# USING GROQ API TO GENERATE JOB DESCRIPTION

In [1]:
#imports
from groq import Groq
import pandas as pd
import time
import os
import codecs

In [2]:
grok_api_key = 'YOUR_API_KEY' #API_KEY is removed, in order to run the code, replace the valid groq api_key

client = Groq(
    api_key=grok_api_key,
)

In [3]:
prompt_required = '''generate a 200 word job description for this resume without proper nouns 
The job description should only include and do not include statements like 'Here is a 200-word job description for the provided resume:': 
A Role summary summarizing the role's responsibilities and objectives
Responsibilities, which should include specific tasks and duties related to the role
Required qualifications, which should include education, certifications, and experience
Desired qualifications, which should include additional education, certifications, and experiences that would be beneficial for the role
Additional requirements, which should include any special requirements or constraints related to the role 
'''

def get_prompt(resume):
    return f'{prompt_required} "{resume}"'

In [4]:
#create a dictionary to store the row data of resume corpus in the required format
data = {"resume": [], "categories": [], "job_description": []}

#create a dataframe and keep adding the rows to dataframe for each file we process
df = pd.DataFrame(data)

# Define the directory
dir_path = "./data/resumes_corpus"

# Get a list of all files in the directory
files = os.listdir(dir_path)

#check the limit of the grop_api and count the number of tokens used
total_tokens = 0

#set the number of files for the training data
num_files = 0


# Iterate over each file
for file in files:

    if file.endswith(".txt"):
        # Construct full file path
        file_path = os.path.join(dir_path, file)
        file_path_label = file_path.replace(".txt", ".lab")
        data = {"resume": [], "categories": [], "job_description": []}

        # print("file_path",file_path)
        # print("file_path_label",file_path_label)


        # Open the file
        with codecs.open(file_path, "r", encoding="utf8", errors="ignore") as f:
            # Read the file's contents
            resume = f.read()
            num_files += 1

            response = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[{"role": "user", "content": get_prompt(resume)}],
                temperature=1,
                max_tokens=1024,
                top_p=1,
                stream=False,
                stop=None,
            )
            message_content = response.choices[0].message.content
            # print(message_content)
            tokens = response.usage.total_tokens
            total_tokens += tokens
            
            data["resume"].append(resume)
            data["job_description"].append(message_content)
            print("tokens",tokens, 'total_tokens',total_tokens)


        with codecs.open(file_path_label, "r", encoding="utf8", errors="ignore") as f:
            # Read the file's contents
            content = f.read()
            content = content.split("\n")
            if "" in content:
                content.remove("")
            data["categories"].append(content)

        new_row_df = pd.DataFrame(data)
        # Add the new row to the DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)
        

        #rate limiter
        if total_tokens > 55000:
            total_tokens = 0
            df.to_json("./data_sample/data.json", orient="records", indent=4)
            # time.sleep(10)

        print(num_files)
        if num_files == 15:
            break



tokens 2136 total_tokens 2136
1
tokens 920 total_tokens 3056
2
tokens 1305 total_tokens 4361
3
tokens 1210 total_tokens 5571
4
tokens 1463 total_tokens 7034
5
tokens 4182 total_tokens 11216
6
tokens 3964 total_tokens 15180
7
tokens 1423 total_tokens 16603
8
tokens 1589 total_tokens 18192
9
tokens 1972 total_tokens 20164
10
tokens 3378 total_tokens 23542
11
tokens 1788 total_tokens 25330
12
tokens 2003 total_tokens 27333
13
tokens 873 total_tokens 28206
14
tokens 2342 total_tokens 30548
15


In [5]:
df.to_json("./data_sample/data.json", orient="records", indent=4)